In [1]:
import matplotlib.pylab as plt
from PIL import Image
from io import BytesIO
import gzip

In [2]:
def extract_packets():
    allpackets = []
    with open('htmlhexdump.txt', 'r') as file:
        packet = ''
        for line in file:
            temp = line.split('  ')
            if len(temp) >= 3:
                if(temp[0].startswith('0000')):
                    packet = ''
                packet += ''.join(temp[1].split(' '))
            if len(temp) == 1:
                allpackets.append(packet)
    return allpackets

In [3]:
def parse_packet(data):
    hexdata = bytes.fromhex(data)
    
    #/* handling frame header */
    ethernet_header_len = 14
    
    #/* handling IP headers */
    ipheader_offset = ethernet_header_len
    ipheader = hexdata[ipheader_offset: ipheader_offset+20]
    ipheader_len = (ipheader[0] & 0x0F)*4
    
    #/* handling TCP segment header */
    tcpheader_offset = ethernet_header_len + ipheader_len
    tcpheader = hexdata[tcpheader_offset: tcpheader_offset+20]
    tcplen = ((tcpheader[12] >> 4) & 0x0F) * 4
    
    httpheader_offset = tcpheader_offset + tcplen
    httpdata = hexdata[httpheader_offset:]
    return httpdata

In [4]:
pkts = extract_packets()
httpdata = b''
for pkt in pkts:
    httpdata += parse_packet(pkt)


offset = httpdata.find(b'\r\n\r\n') + 4
mydata = httpdata[offset:]

In [5]:
print("Length of the compressed data is : ", len(mydata), " bytes")

Length of the compressed data is :  34066  bytes


In [6]:
print("Length of the uncompresed data is : ", len(gzip.decompress(mydata)), " bytes")

Length of the uncompresed data is :  171296  bytes


In [7]:
print("Ratio of decompressed data to compressed data is : ", len(gzip.decompress(mydata))/len(mydata))

Ratio of decompressed data to compressed data is :  5.028356719309576
